In [ ]:
# basic
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True) #不要科學記號
pd.set_option('display.float_format',lambda x : '%.4f' % x)
pd.set_option('display.max_rows', 10)

# 獲得股票資訊
import pandas_datareader as pdr
import yfinance as yf

# 繪圖用的套件
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#獲得時間
import datetime as datetime
import time

#talib計算金融技術指標
import talib
from talib import abstract

import requests

today = datetime.datetime.now() 
day_ago = today - datetime.timedelta(days=100)

import pandas as pd
from datetime import date
import pymssql 

# 連接資料庫

In [82]:
conn = pymssql.connect(host='192.168.137.1',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
cursor = conn.cursor()
sql = 'select * from UserList'
df_user = pd.read_sql(sql, conn)     # 1 UserList
sql = 'select * from StrategyList'
df_stg = pd.read_sql(sql, conn)      # 2 StrategyList
sql = 'select * from TodaySignal'
df_signal = pd.read_sql(sql, conn)   # 3 TodaySignal

In [83]:
df_user

,ID,Name,0,1,2
0,u3f83uhd,Emery,0,1,0
1,bsbw4gwe,Sandy,1,0,1
2,svadvvw,Kevin,1,0,1
3,g43egsde,Ken,1,0,1
4,fsavsv,Jack,1,0,1


In [84]:
df_stg

,StrategyName,Description,Parameter
0,BBAND,今日股價由下而上 穿過下緣,"BBANDS(df,timeperiod=20,nbdevup=1.0,nbdevdn=1...."
1,RSI,RSI小於30，處於超賣區,"RSI(df,timeperiod=14) ..."
2,MACD,快線向上突破慢線,"MACD(df,fastperiod=12,slowperiod=26,signalperi..."


In [85]:
df_signal.head(5)

,StockSymbol,StockName,IndustryType,0,1,2
0,1101,台泥,水泥工業,0,0,0
1,1102,亞泥,水泥工業,0,0,0
2,1103,嘉泥,水泥工業,0,0,0
3,1104,環泥,水泥工業,0,0,0
4,1108,幸福,水泥工業,0,0,0


調整 StrategyList 的數值，並再存成一些 list

In [86]:
stg_name = df_stg['StrategyName'].tolist()
stg_name = [str(x.strip()) for x in stg_name]  #去除空白!!
stg_par = df_stg['Parameter'].tolist() 
stg_par = [str(x.strip()) for x in stg_par]
stg_index = [str(x) for x in df_stg.index.tolist()]
print(stg_name,stg_par)

['BBAND', 'RSI', 'MACD'] ['BBANDS(df,timeperiod=20,nbdevup=1.0,nbdevdn=1.0,matype=0)', 'RSI(df,timeperiod=14)', 'MACD(df,fastperiod=12,slowperiod=26,signalperiod=9)']


In [95]:
#sql = """
#          INSERT INTO UserList (ID,Name,[0],[1],[2]) 
#          VALUES(%s,%s,%s,%s,%s)
#        """
#cursor.execute(sql, ('U53ba0f86ac04064b808aa924e057947b', 'Andy', '1', '0', '1'))
#conn.commit()

# 取得股票代號

In [96]:
def stk_symbol():
    res = requests.get("https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y")
    df = pd.read_html(res.text)[0]
    df.drop(columns=[0,1,4,5,7,8,9], inplace=True) 
    df.columns = df.iloc[0]
    df.rename(columns = {'有價證券代號':'股票代號','有價證券名稱':'股票名稱'} , inplace = True)
    df = df.iloc[1:]
    df.reset_index(inplace = True)
    df.drop(columns=['index'], inplace=True) 
    return df
df_stk = stk_symbol()
df_stk.head()

,股票代號,股票名稱,產業別
0,1101,台泥,水泥工業
1,1102,亞泥,水泥工業
2,1103,嘉泥,水泥工業
3,1104,環泥,水泥工業
4,1108,幸福,水泥工業


# 一、跑每日策略

原本 : BBAND20 = abstract.BBANDS(df, timeperiod=20, nbdevup=1, nbdevdn=1, matype=2)

In [97]:
def StockData(stock_symbol):
    df = yf.Ticker(stock_symbol).history(period='1d', start=day_ago, end=today)
    df.rename(columns={'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume': 'volume', 'Dividends': 'dividends'}, inplace=True)
    df = df.astype('float')

    # ta_list = talib.get_functions() 能查看全部指標
    ta_list = stg_par
    
    for x in ta_list:
        try:
            # x 為技術指標的代碼，透過迴圈填入，再透過 eval 計算出 output
            output = eval('abstract.'+ x )
            # 如果輸出是一維資料，幫這個指標取名為 x 本身；多維資料則不需命名
            output.name = x.lower() if type(output) == pd.core.series.Series else None
            # 透過 merge 把輸出結果併入 df DataFrame
            df = pd.merge(df, pd.DataFrame(output), left_on = df.index, right_on = output.index)
            df = df.set_index('key_0')
        except:
            print('Error:', x)
    df.reset_index(inplace = True)
    return df

In [98]:
# 測試單一標的 2330 丟進去 StockData()，應增加技術指標算出來的欄位
df_index = StockData('2330.TW')
df_index.head()

,key_0,open,high,low,close,volume,dividends,Stock Splits,upperband,middleband,lowerband,"rsi(df,timeperiod=14)",macd,macdsignal,macdhist
0,2021-03-10,596.0756,596.0756,588.1412,592.1084,40146301.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-11,591.1166,606.9855,590.1248,604.0101,54416046.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-12,609.9609,612.9363,604.0100,608.9691,33168846.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-15,608.9691,608.9691,604.0100,605.9937,27416758.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-03-16,605.0019,607.9773,602.0265,607.9773,36949425.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 每日訊號

    signal() 就是把所有標的(953個)，迴圈丟入 StockData()，計算出技術指標。

    並加入策略，判斷今日是否有買入信號。

    BBAND買入信號 : 當今日收盤價由下而上，穿過布林通道下緣。昨日布林下緣 > 昨日收盤價 且 今日布林下緣 < 今日收盤價         
    
    RSI買入信號 : RSI < 30，處於超賣區
    
    MACD買入信號 : 快線向上突破慢線。昨天快線 < 昨天慢線 且 今日快線 > 今日慢線                  

In [ ]:
def signal():
    signal_list = []
    for i in range( len(df_stk) ):
        one_list = []
        try:
            df_index = StockData(df_stk['股票代號'].loc[i]+'.TW')
            today = df_index.loc[len(df_index)-1]
            yesterday = df_index.loc[len(df_index)-2]
            
            # 布林通道策略:今日股價由下而上 穿過下緣
            if ( yesterday['lowerband'] > yesterday['close'] ) and ( today['lowerband'] < today['close'] ):
                print(df_stk['股票代號'].loc[i], '布林:收盤價由下而上，穿過下緣')
                one_list.append(1)
            else:
                one_list.append(0)   
            # RSI策略:
            if ( yesterday['rsi(df,timeperiod=14)'] < 30 ):
                print(df_stk['股票代號'].loc[i], 'RSI:小於30，處於超賣區')
                one_list.append(1)
            else:
                one_list.append(0)  
            # MACD策略:
            if ( yesterday['macd'] < yesterday['macdsignal'] ) and ( today['macd'] > today['macdsignal'] ):
                print(df_stk['股票代號'].loc[i], 'MACD:快線向上突破慢線')
                one_list.append(1)
            else:
                one_list.append(0)
            signal_list.append(one_list)
        except:
            signal_list.append([0,0,0])
            print(df_stk['股票代號'].loc[i], 'error')
    return signal_list

In [ ]:
a = signal()

In [35]:
df_a = pd.DataFrame(a, columns = stg_name)
df_all_index = pd.concat([df_stk, df_a], axis=1)
df_all_index

,股票代號,股票名稱,產業別,BBAND,RSI,MACD
0,1101,台泥,水泥工業,0,0,0
1,1102,亞泥,水泥工業,0,0,0
2,1103,嘉泥,水泥工業,0,0,0
3,1104,環泥,水泥工業,0,0,0
4,1108,幸福,水泥工業,0,0,0
...,...,...,...,...,...,...
948,9944,新麗,其他業,0,0,0
949,9945,潤泰新,其他業,0,0,0
950,9946,三發地產,建材營造業,0,0,0
951,9955,佳龍,其他業,0,0,0


## 每日訊號-寫入資料庫

In [39]:
# 先清空資料表
conn = pymssql.connect(host='192.168.137.1',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
cursor = conn.cursor()
sql = """
      TRUNCATE TABLE TodaySignal;
    """
cursor.execute( sql )
conn.commit()
print('...Clear Up')

...Clear Up


In [40]:
# df_all_index 全部股票買入訊號的表 寫入資料庫
for index, row in df_all_index.iterrows():
    try:
        sql = """
          INSERT INTO TodaySignal (StockSymbol,StockName,IndustryType,[0],[1],[2]) 
          VALUES(%s,%s,%s,%s,%s,%s)
        """
        cursor.execute(sql, (row.股票代號, row.股票名稱, row.產業別, row.BBAND, row.RSI, row.MACD))
        conn.commit()

    except:
        print('Error:', row.股票代號, ' ', row.股票名稱, ' ', row.產業別)

cursor.close()

In [41]:
# 檢查是否寫入
conn = pymssql.connect(host='192.168.137.1',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
df_signal = pd.read_sql('select * from TodaySignal', conn)   # 3 TodaySignal
df_signal

,StockSymbol,StockName,IndustryType,0,1,2
0,1101,台泥,水泥工業,0,0,0
1,1102,亞泥,水泥工業,0,0,0
2,1103,嘉泥,水泥工業,0,0,0
3,1104,環泥,水泥工業,0,0,0
4,1108,幸福,水泥工業,0,0,0
...,...,...,...,...,...,...
948,9944,新麗,其他業,0,0,0
949,9945,潤泰新,其他業,0,0,0
950,9946,三發地產,建材營造業,0,0,0
951,9955,佳龍,其他業,0,0,0
